In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir1 = 'log/20210302_193140_mcmc_rasch/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [3]:
# log_dir3 = 'log/20210302_203421_rash01/'
log_dir3 = 'log/20210302_211048_rash01/'
exp_data3 = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)


99


In [4]:
exp_data3.raw_data['beta']

array([1., 1., 1., 1., 1., 1.])

## Plot weighted histograms

In [5]:
particles = load_obj('particles', log_dir3)
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
particles.weights

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
particles.get_acceptance_rate_for_particle_m(0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
corrs = load_obj('jitter_corrs', log_dir3)
corrs

{'beta': array([-1.93208479e-02,  2.12968283e-01,  0.00000000e+00,  4.73750417e-01,
         5.85032731e-01,  0.00000000e+00,  5.76715152e-01,  0.00000000e+00,
         3.21090982e-01,  0.00000000e+00,  6.66624935e-17, -8.83868001e-16,
         0.00000000e+00,  3.06161631e-01,  0.00000000e+00,  2.63130961e-01,
         4.50291220e-01,  0.00000000e+00,  3.64953192e-01,  1.73845703e-01,
         0.00000000e+00,  5.50739890e-01,  2.82917927e-01,  0.00000000e+00,
         3.02262305e-01,  0.00000000e+00,  0.00000000e+00,  5.06555705e-01,
         0.00000000e+00,  0.00000000e+00,  2.97174513e-01,  9.19786788e-02,
         0.00000000e+00, -1.43136177e-02,  5.94880878e-01,  6.51310212e-01,
         0.00000000e+00,  2.82750636e-02,  0.00000000e+00,  3.25604427e-01,
         1.64221364e-01,  4.51908608e-01,  0.00000000e+00,  0.00000000e+00,
         6.08179364e-01,  0.00000000e+00,  4.91405350e-01,  5.43887807e-01,
         0.00000000e+00,  4.61829154e-01,  5.76819884e-01,  0.00000000e+00,
    

In [8]:
# particles = load_obj('particles', log_dir3)
# particles.resample_particles_bundles()
# ps_smc2 = dict()
# for name in particles.param_names:
#     ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


##

## Or load existing directory

In [9]:
ps_mcmc = post_process_sign(ps_mcmc)
ps_smc2 = post_process_sign(ps_smc2)


## Plot MCMC samples

In [18]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df = df[df.row==0]
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3 = df3[df3.row==0]
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100) 

alt.Chart(...)

In [11]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

## Plot quantiles

In [12]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df[df.row==0]).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [13]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [14]:
plot_data = plot_data[plot_data['index']=='r_0.c_0']

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)


## chart overlaying quantiles with real data

In [15]:
param = 'alpha'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [16]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['alpha'], columns=['data'])
dd['col'] = np.arange(6)
dd['row'] = 0
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [17]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-3,3])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)